#### **Importing a CSV file into a postgres database:** 


##### Steps:
- Import the csv file into a pandas df.
- Clean the table name and remove all extra symbols.
- clean the column headers and remove all extra symbols, spaces, capital letters.
- write the create table SQL statement
- Import the data into the Database.

In [ ]:
# Import all needed libraries
import pandas as pd
import os
import numpy as numpy
import psycopg2

##### **Find CSV files in directory:**
- Find the cvs files in my current working directory.
- Isolate only the csv files.
- Make a new directory.
- Move the CSV files in the new directory.

In [ ]:
# List all files in my current working directory
os.listdir(os.getcwd())

# Lets isolate the csv files from my folder
csv_files=[]
for file in os.listdir(os.getcwd()):
    if file.endswith(".csv"):
        csv_files.append(file)
csv_files

In [ ]:
# Make a new directory
datasets_file=input("Please enter the folder to save CSV file?")

# Create the bash or cmd command to make a new directory
mkdir= "mkdir {}".format(datasets_file)
os.system(mkdir)

In [ ]:
# Move the csv files in the new directory
for csv in csv_files:
    mv_file="move {} {}".format(csv,datasets_file)
    # mv_file="mv {} {}".format(csv,dataset_dir) shell command linux
    os.system(mv_file)
    print(mv_file)

##### **Create the Pandas df from the CSV files:**

In [ ]:
# Here a very good way to store dataframes in dictionary:
data_path=os.getcwd()+ "\\" +datasets_file+"\\"
# print(data_path)
df={}
for file in csv_files:
    try:
        df[file]=pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file]=pd.read_csv(data_path+file,encoding="ISO-8859-1")


In [112]:
df["population_by_zip_2010.csv"]

,population,minimum_age,maximum_age,gender,zipcode,geo_id
0,50,30.0,34.0,female,61747,8600000US61747
1,5,85.0,NaN,male,64120,8600000US64120
2,1389,30.0,34.0,male,95117,8600000US95117
3,231,60.0,61.0,female,74074,8600000US74074
4,56,0.0,4.0,female,58042,8600000US58042
...,...,...,...,...,...,...
1622826,66,15.0,17.0,female,28640,8600000US28640
1622827,791,25.0,29.0,male,98604,8600000US98604
1622828,55,55.0,59.0,female,29545,8600000US29545
1622829,10,25.0,29.0,female,45319,8600000US45319


In [ ]:
# replacment dictionary that maps pandas dtypes to sql dtypes
replacments={
    "object":"varchar",
    "float64":"float",
    "int64":"int"
}

##### **Clean Table names and Column names :**

In [ ]:
for k in csv_files:
    dataframe=df[k]    
    clean_table_name=k.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
    # remove .csv extension from clean_table_name
    table_name="{}".format(clean_table_name.split(".")[0])
    print(table_name)
    # Clean the column name
    dataframe.columns=[x.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
        for x in dataframe.columns      
        ]
    print(dataframe.columns)
    

In [ ]:
# String of creating table schema
# table schema
col_str=", ".join(["{} {}".format(c,d) for (c,d) in zip(dataframe.columns, \
                                        dataframe.dtypes.replace(replacments))])
print(col_str)
    

##### **Adding the Database connection:**

In [ ]:
# open a database connection
host="localhost"
database="mydb"
user="postgres"
password="password"
port=5432

conn_string= "host=%s database=%s user=%s password=%s port=%s" % \
                (host,database,user,password,port )
conn=psycopg2.connect(conn_string)
cur=conn.cursor()
print("connected to database successfully")


In [ ]:
# drop table with the same name
cur.execute("DROP TABLE IF EXISTS %s" % (table_name))

In [ ]:
# Create table
cur.execute("create table %s (%s) "% (table_name,col_str))


In [ ]:
# Saving the Dataframe in csv file a:
dataframe.to_csv(k,header=dataframe.columns,index=False,encoding="utf-8")
# open the csv file , save it as an object
my_file=open(k)

In [ ]:
# How to upload the csv file into the database
# We can use the copy method
SQL_STATEMENT="""
Copy %s FROM STDIN WITH
    CSV
    HEADER
    DELIMETER AS ","
"""
cur.copy_expert(sql=SQL_STATEMENT % table_name,file=my_file)
print("File copied to database!")

In [ ]:
# Grant access to users
cur.execute(f"GRANT SELECT ON TABLE {table_name} TO PUBLIC")

In [ ]:
# Full
for k in csv_files:
    dataframe=df[k]    
    clean_table_name=k.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
    # remove .csv extension from clean_table_name
    table_name="{}".format(clean_table_name.split(".")[0])
    print(table_name)
    # Clean the column name
    dataframe.columns=[x.lower().replace(" ","_").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
        for x in dataframe.columns      
        ]
    print(dataframe.columns)
    
    # table schema
    col_str=", ".join(["{} {}".format(c,d) for (c,d) in zip(dataframe.columns, \
                                        dataframe.dtypes.replace(replacments))])
    print(col_str)
    
    
    # connecting to our database
    # open a database connection
    host="localhost"
    database="mydb" 
    user="postgres"
    password="admin"
    port=5432

    conn_string= "host=%s dbname=%s user=%s password=%s port=%s" % \
                    (host,database,user,password,port )
    conn=psycopg2.connect(conn_string)
    cur=conn.cursor()
    print("connected to database successfully")
    cur.execute("DROP TABLE IF EXISTS %s" % (table_name))
    # Create table
    cur.execute("create table %s (%s) "% (table_name,col_str))
    print("{} was created successfully!".format(table_name))
    
    # Saving the Dataframe in csv file:
    dataframe.to_csv(k,header=dataframe.columns,index=False,encoding="utf-8")
    # k is here my new file name
    my_file=open(k)
    print("File is open as object...")
    SQL_STATEMENT="""
        Copy %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
        """
    cur.copy_expert(sql=SQL_STATEMENT % table_name,file=my_file)
    print("File copied to database!")       
    
    conn.commit()
    cur.close()
    

In [116]:
n="Yosue  f%?"
clean_table_name=n.lower().replace(" ","-").replace("?","") \
        .replace("-","_").replace( "/" ,"_").replace("\\","_") \
        .replace("%","").replace(")","").replace("(","").replace("$","")
clean_table_name

'yosue__f'